In [24]:
from multiprocessing import Pool
from tqdm.auto import tqdm
import cProfile, pstats
import numpy as np
from numba import njit, prange
%load_ext snakeviz

In [3]:
#test pool with gen
items = ['a','b','c']
vals = (['a',[100,101,102]],['b',[200,201,202,203]],['c',[300,301]]) # list-based
j = iter(range(len(vals)))
k={}


In [4]:
def test(item):
    kk = next(j)
    k[kk] = item
    for val in enumerate(item[1]):
        print(f'\n{item[0]}[{i}] has {val}')

In [25]:
@njit(parallel=True)
def test_nb(item):
    for val in prange(item.shape[0]):
        print(f'\n {val}')

In [5]:
def dtest(items):
    print(f'\n[{items[0]}] has {items[1]}')

In [6]:
vs= {k:v for k,v in vals}
v_ks = {i:k for i,k in enumerate(vs.keys())}
val_count = sum( len(v) for v in list(vs.values()))
val_count

9

In [7]:
ixn_keys = [[(ik,nk) for nk,_ in enumerate(vs[ v_ks[ik] ])] for ik in tuple(v_ks.keys())]
dummy_dict = {}
for inkl, nl in zip( ixn_keys,vs.values() ):
    print(f'associate members of {nl} to {inkl}')
    for ink,n in zip(inkl,nl):
        print(f'{ink} <-- {n}')
        dummy_dict[ink] = n #dict-based

associate members of [100, 101, 102] to [(0, 0), (0, 1), (0, 2)]
(0, 0) <-- 100
(0, 1) <-- 101
(0, 2) <-- 102
associate members of [200, 201, 202, 203] to [(1, 0), (1, 1), (1, 2), (1, 3)]
(1, 0) <-- 200
(1, 1) <-- 201
(1, 2) <-- 202
(1, 3) <-- 203
associate members of [300, 301] to [(2, 0), (2, 1)]
(2, 0) <-- 300
(2, 1) <-- 301


In [8]:
(a,b), c = list(dummy_dict.items())[0]
a,b,c

(0, 0, 100)

In [26]:
def test_():
    pool = Pool()
    with pool:
        tuple( tqdm( pool.imap( test_nb, vals ), total=3) )

In [10]:
def dtest_():
    pool = Pool()
    with pool:
        tuple( tqdm( pool.imap( dtest, dummy_dict.items() ), total=3) )

In [11]:
def profile_f(f):
    profiler = cProfile.Profile()
    profiler.enable()
    f()
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('tottime')
    return stats

In [13]:
# lt = profile_f(test_).print_stats()
# dt = profile_f(dtest_).print_stats()


In [27]:
%snakeviz dtest_()

  0%|          | 0/3 [00:00<?, ?it/s]


[(0, 1)] has 101
[(0, 0)] has 100

[(1, 1)] has 201


[(1, 2)] has 202

[(1, 0)] has 200
[(1, 3)] has 203

[(2, 0)] has 300
[(0, 2)] has 102

[(2, 1)] has 301




9it [00:00, 249.83it/s]              


 
*** Profile stats marshalled to file '/tmp/tmp696161vu'. 
Embedding SnakeViz in this document...
